In [10]:
# Step 1: Imports
import tensorflow as tf
import os

# Step 2: Load the trained model
model = tf.keras.models.load_model("../models/best_model.h5", compile=False)

# Step 3: Convert to TFLite (Basic)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Step 4: Save the TFLite model (basic version)
os.makedirs("../models", exist_ok=True)
with open("../models/model_fp32.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Basic float32 TFLite model saved (no quantization).")

# Step 5: Convert to TFLite with Dynamic Range Quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Step 6: Save quantized model
with open("../models/model_quant.tflite", "wb") as f:
    f.write(tflite_quant_model)

print("✅ Quantized TFLite model saved (dynamic range).")

# Optional: Show size comparison
import os
fp32_size = os.path.getsize("../models/model_fp32.tflite") / 1024
quant_size = os.path.getsize("../models/model_quant.tflite") / 1024

print(f"\n📦 Float32 TFLite size: {fp32_size:.2f} KB")
print(f"📦 Quantized TFLite size: {quant_size:.2f} KB")
print("🎯 Compression ratio:", round(fp32_size / quant_size, 2))


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmph1y84vea\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmph1y84vea\assets


Saved artifact at 'C:\Users\hp\AppData\Local\Temp\tmph1y84vea'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 35), dtype=tf.float32, name=None)
Captures:
  2570117207792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117212720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117242672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117241616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117248656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117337280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117346784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117347488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117535472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117536176: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Basic floa

INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmp_2cfsdna\assets


Saved artifact at 'C:\Users\hp\AppData\Local\Temp\tmp_2cfsdna'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 35), dtype=tf.float32, name=None)
Captures:
  2570117207792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117212720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117242672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117241616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117248656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117337280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117346784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117347488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117535472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2570117536176: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Quantized 

In [11]:
# Test the .tflite Model

import tensorflow as tf
import numpy as np

# Load test data
X_test = np.load("../data/processed/X_test.npy")

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="../models/model_quant.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a sample image (batch of 1)
img = X_test[0:1].astype(np.float32)

# Run inference
interpreter.set_tensor(input_details[0]['index'], img)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Get predicted class
predicted_class = np.argmax(output_data)
print("Predicted Class Index:", predicted_class)


Predicted Class Index: 31


e:\DataScienceProjects\isl_translator\isl_env\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [12]:
 # Show Class Label

import joblib
label_encoder = joblib.load("../logs/label_encoder.pkl")
class_label = label_encoder.inverse_transform([predicted_class])[0]
print("Predicted Label:", class_label)


Predicted Label: 31


Lighter size for mobile 

In [ ]:
# Float16 Quantization (Best for GPU/mobile inference)

import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model("../models/best_model.h5", compile=False)

# Create a converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply float16 quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# Convert the model
tflite_model = converter.convert()

# Save to file
with open("../models/model_fp16.tflite", "wb") as f:
    f.write(tflite_model)

print(" Float16 Quantized TFLite model saved.")


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpglgdl5x2\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpglgdl5x2\assets


Saved artifact at 'C:\Users\hp\AppData\Local\Temp\tmpglgdl5x2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 35), dtype=tf.float32, name=None)
Captures:
  2568391833232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391805584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391818608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391814032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391916208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391912688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392085696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392086400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392095376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392096080: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Float16 Qu

In [14]:
# Full Integer Quantization (INT8) –  maximum compression

#  Define Representative Dataset Function
import numpy as np

X_test = np.load("../data/processed/X_test.npy")

def representative_data_gen():
    for i in range(100):  # Use ~100 samples for calibration
        yield [X_test[i:i+1].astype(np.float32)]


#Perform Full INT8 Conversion

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable full integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Force all ops to be int8
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert
tflite_int8_model = converter.convert()

# Save
with open("../models/model_int8.tflite", "wb") as f:
    f.write(tflite_int8_model)

print(" INT8 Quantized TFLite model saved.")


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmp4j_pn_ku\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmp4j_pn_ku\assets


Saved artifact at 'C:\Users\hp\AppData\Local\Temp\tmp4j_pn_ku'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 35), dtype=tf.float32, name=None)
Captures:
  2568391833232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391805584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391818608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391814032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391916208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568391912688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392085696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392086400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392095376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2568392096080: TensorSpec(shape=(), dtype=tf.resource, name=None)


e:\DataScienceProjects\isl_translator\isl_env\lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


 INT8 Quantized TFLite model saved.
